# MSD - tutorial_3

In [1]:
import sys
ver_info = sys.version
print("This jupyter notebook was executed with:\n{0}".format(ver_info));


This jupyter notebook was executed with:
2.7.14 |Anaconda, Inc.| (default, Nov  8 2017, 13:40:45) [MSC v.1500 64 bit (AMD64)]


## Imports and path to dataset

In [2]:
# usual imports 
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np

In [4]:
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='../MillionSongSubset'  # mine is in the same directory as the pynb file (rshn) 
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
print(msd_subset_path)
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check



# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='../MSongsDB'
print(msd_code_path)
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# imports specific to the MSD
import hdf5_getters as GETTERS

# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))


../MillionSongSubset
../MSongsDB


Now starting tutorial 3: 

In [6]:
def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'"+s.replace("'","''")+"'"

In [25]:
# PATH TO track_metadat.db
# CHANGE THIS TO YOUR LOCAL CONFIGURATION
# IT SHOULD BE IN THE ADDITIONAL FILES
# (you can use 'subset_track_metadata.db')
dbfile = '../MSongsDB/Tasks_Demos/SQLite/track_metadata.db'
dbfile_subset = '../MillionSongSubset/AdditionalFiles/subset_track_metadata.db'


Connect to DB

In [26]:
# connect to the SQLite database
conn = sqlite3.connect(dbfile_subset)

# from that connection, get a cursor to do queries
c = conn.cursor()

# conn.close()

In [27]:
# so there is no confusion, the table name is 'songs'
TABLENAME = 'songs'

## General SQLITE Demo

In [28]:
# list all tables in that dataset
# note that sqlite does the actual job when we call fetchall() or fetchone()
q = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
res = c.execute(q)
print "* tables contained in that SQLite file/database (should be only 'songs'):"
print res.fetchall()


* tables contained in that SQLite file/database (should be only 'songs'):
[(u'songs',)]


In [39]:
# list all columns names from table 'songs'
q = "SELECT sql FROM sqlite_master WHERE tbl_name = 'songs' AND type = 'table'"
res = c.execute(q)
print '* get info on columns names (original table creation command):'
print res.fetchall()[0][0]

* get info on columns names (original table creation command):
CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int)


In [40]:
# list all indices
q = "SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='songs' ORDER BY name"
res = c.execute(q)
print '* one of the index we added to the table to make things faster:'
print res.fetchone()


* one of the index we added to the table to make things faster:
(u'idx_artist_id',)


In [48]:
# find the PRIMARY KEY of a query
# by default it's called ROWID, it would have been redefined if our primary key
# was of type INTEGER
q = "SELECT ROWID FROM songs WHERE artist_name='The Beatles'"
res = c.execute(q)
print '* get the primary key (row id) of one entry where the artist is The Beatles:'
print res.fetchone()


* get the primary key (row id) of one entry where the artist is The Beatles:
(1040,)


In [54]:
# find an entry with The Beatles as artist_name
# returns all info (the full table row)
q = "SELECT * FROM songs WHERE artist_name='The Beatles' LIMIT 1"
res = c.execute(q)
print '* get all we have about one track from The Beatles:'
print res.fetchone()


* get all we have about one track from The Beatles:
(u'TRAHSSO128EF347345', u'Derek Taylor - Introduction', u'SORTPSA12A67ADBFE2', u'Here There And Everywhere', u'AR6XZ861187FB4CECD', u'b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d', u'The Beatles', 53.78567, 0.840409662154, 0.840462688027, 0)


Playing around now...

In [55]:
# Query to get total number of rows in 'songs' 
q = "SELECT count() FROM songs"
res = c.execute(q)
print res.fetchall()

[(10000,)]


In [64]:
# query to get a couple artist names

q = "SELECT artist_name from songs"
res = c.execute(q)
result = res.fetchall()

artist_name_list = set()

for item in result: 
    artist_name_list.add(item[0])
    
# Print the first 10 artists    
print list(artist_name_list)[0:10]

[u'Alfredo Kraus - Orquesta de Conciertos de Madrid - Pablo Soroz\xe1bal', u'Pale Forest', u'The Real Kids', u'JennyAnyKind', u'Little Willie John', u'Spooky Tooth / Mike Harrison', u'T-Bone', u'4 Skins', u'089 Clique feat. Prophet', u'Graham Central Station']


In [78]:
# query to get the top 10 hotttnesss tracks 
q = "SELECT artist_hotttnesss FROM songs ORDER BY artist_hotttnesss DESC LIMIT 10"
res = c.execute(q)
print res.fetchall()

[(1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.08250255673,), (1.02125558749,)]


## Demos around ARTIST_ID

In [83]:
# query for all the artists Echo Nest ID
# the column name is 'artist_id'
# DISTINCT makes sure you get each ID returned only once
q = "SELECT DISTINCT artist_id, artist_name FROM " + TABLENAME
res = c.execute(q)
artists = res.fetchall() # does the actual job of searching the db
print '* found',len(artists),'unique artist IDs, response looks like:'
print artists[:3]

# oh okay, another way to grab all artists.  forgot about the distinct thing 

* found 4437 unique artist IDs, response looks like:
[(u'AR009211187B989185', u'Carroll Thompson'), (u'AR00A6H1187FB5402A', u'The Meatmen'), (u'AR00LNI1187FB444A5', u'Bruce BecVar')]


In [84]:
# more cumbersome, get unique artist ID but with one track ID for each.
# very usefull, it gives you a HDF5 file to query if you want more
# information about this artist
q = "SELECT artist_id,track_id FROM songs GROUP BY artist_id"
res = c.execute(q)
artist_track_pair = res.fetchone()

print '* one unique artist with some track (chosen at random) associated with it:'
print artist_track_pair



* one unique artist with some track (chosen at random) associated with it:
(u'AR009211187B989185', u'TRBFYQS128F92E83A0')


In [88]:
# get artists having only one track in the database
q = "SELECT artist_id, artist_name, track_id, title FROM songs GROUP BY artist_id HAVING ( COUNT(artist_id) = 1 )"
q += " ORDER BY RANDOM()"
res = c.execute(q)
artist_track_pair = res.fetchone()
print '* one artist that has only one track in the dataset:'
print artist_track_pair


* one artist that has only one track in the dataset:
(u'ARVFDXZ1241B9C8865', u'Pedro Garcia & His Del Prado Orchestra', u'TRBDDCT12903CEF57E', u'Policias Y Ladrones')


In [89]:
# get artists with no musicbrainz ID
# of course, we want only once each artist
# for demo purpose, we ask for only two at RANDOM
q = "SELECT artist_id,artist_mbid FROM songs WHERE artist_mbid=''"
q += " GROUP BY artist_id ORDER BY RANDOM() LIMIT 2"
res = c.execute(q)
print '* two random unique artists with no musicbrainz ID:'
print res.fetchall()



* two random unique artists with no musicbrainz ID:
[(u'ARDXGVT124207832C0', u''), (u'ARGCNAB1242077FED1', u'')]


## Demos around names

In [98]:
# get all tracks by artist The Beatles
# artist name must be exact!
# the encode_string function simply deals with ' (by doubling them)
# and add ' after and before the string.
q = "SELECT artist_name, track_id FROM songs WHERE \
    ( artist_name = 'The Beatles' OR artist_name= 'Pedro Garcia & His Del Prado Orchestra')"
# q += encode_string('The Beatles')
res = c.execute(q)
print "* two track id from 'The Beatles', found by looking up the artist by name:"
print res.fetchall()[:4]



* two track id from 'The Beatles', found by looking up the artist by name:
[(u'Pedro Garcia & His Del Prado Orchestra', u'TRBDDCT12903CEF57E'), (u'The Beatles', u'TRAHSSO128EF347345')]


In [99]:
# we find all release starting by letter 'T'
# T != t, we're just looking at albums starting with capital T
# here we use DISTINCT instead of GROUP BY artist_id
# since its fine that we find twice the same artist, as long as it is not
# the same (artist,release) pair
q = "SELECT DISTINCT artist_name,release FROM songs WHERE SUBSTR(release,1,1)='T'"
res = c.execute(q)
print '* one unique artist/release pair where album starts with capital T:'
print res.fetchone()


* one unique artist/release pair where album starts with capital T:
(u'23rd Turnoff', u'The Psychedelic Scene')


# Demos about floats

In [102]:
# get all artists whose artist familiarity is > .8
q = "SELECT DISTINCT artist_name, artist_familiarity FROM songs WHERE artist_familiarity>.8"
res = c.execute(q)
print '* one artist having familiaryt >0.8:'
# print res.fetchone()
print res.fetchall()[:5]


* one artist having familiaryt >0.8:
[(u'3 Doors Down', 0.840432466617), (u'3 Doors Down', 0.840432421742), (u'30 Seconds To Mars', 0.909324833658), (u'50 Cent', 0.883487079775), (u'50 Cent / Mobb Deep', 0.88348708528)]


In [112]:
# get one artist with the highest artist_familiarity but no artist_hotttnesss
# notice the alias af and ah, makes things more readable
q = "SELECT DISTINCT artist_name, artist_familiarity as af, artist_hotttnesss as ah"
q += " FROM songs WHERE NOT ah=0 ORDER BY af"
res = c.execute(q)
print '* get the artist with the highest familiarity that has no computed hotttnesss:'
# print res.fetchone()
print res.fetchall()



* get the artist with the highest familiarity that has no computed hotttnesss:
[(u'John Hammond', 0.0, 0.161319550223), (u'Starless & Bible Black', 0.0, 0.251599923727), (u'Bas-Sheva', 0.0, 0.285498363109), (u'Billy Price_ Fred Chapellier', 0.0, 0.293089232126), (u'Wanda De Sah', 0.0, 0.296426618229), (u'Christoper Titus', 0.0, 0.370703169225), (u'John Davis & Emma Doyle', 0.0, 0.387635477706), (u'Montserrat Caball\xe9/Royal Philharmonic Orchestra/Lamberto Gardelli', 0.0, 0.391370587218), (u'Willie Isz', 0.0, 0.399098557715), (u'Monika_ Wolfgang', 0.0, 0.457072636624), (u'Cassie', 0.0, 0.476803353534), (u'Michael Card', 0.015557880683, 0.344246580364), (u'Pierre Perret', 0.0185015424026, 0.35297647202), (u'Rocky Whatule', 0.0185550930129, 0.0501876394119), (u'Yuko Kanzaki', 0.0210826560279, 0.165342237781), (u'The Presets', 0.0235041920565, 0.524449470713), (u'Patrizia Pace/Waltraud Meier/Frank Lopardo/James Morris/Swedish Radio Choir/Stockholm Chamber Choir/Berliner Philharmoniker/Ric

In [ ]:
# close the cursor and the connection
# (if for some reason you added stuff to the db or alter
#  a table, you need to also do a conn.commit())
c.close()
conn.close()